In [ ]:
# default_exp cf_explainer

In [ ]:
# hide
%load_ext autoreload
%autoreload 2
from ipynb_path import *

In [ ]:
# export
from counternet.import_essentials import *
from counternet.utils import CategoricalNormalizer, flip_binary

Global seed set to 31


In [ ]:
# export
class ExplainerBase(ABC):
    @abstractmethod
    def generate_cf(self, x: torch.Tensor):
        pass

In [ ]:
# export
class LocalExplainerBase(nn.Module, ExplainerBase):
    def __init__(self, pred_fn: Callable[[torch.Tensor], torch.Tensor], configs: Dict[str, Any]={}):
        super().__init__()
        self.pred_fn = pred_fn
        self.configs = configs

    def forward(self):
        raise NotImplementedError

    def compute_regularization_loss(self):
        cat_idx = len(self.model.continous_cols)
        regularization_loss = 0.
        for i in range(self.n_cfs):
            for col in self.model.cat_arrays:
                cat_idx_end = cat_idx + len(col)
                regularization_loss += torch.pow((torch.sum(self.cf[i][cat_idx: cat_idx_end]) - 1.0), 2)
        return regularization_loss

    def configure_optimizers(self):
        return torch.optim.Adam([self.cf], lr=0.001)

In [ ]:
# export
class GlobalExplainerBase(ExplainerBase):
    pass

In [ ]:
# export
class VanillaCF(LocalExplainerBase):
    def __init__(self, pred_fn: Callable[[torch.Tensor], torch.Tensor],
                       cat_normalizer: CategoricalNormalizer=None, configs: Dict[str, Any]={}):
        """vanilla version of counterfactual generation
            - link: https://doi.org/10.2139/ssrn.3063289
        Args:
        """
        super().__init__(pred_fn, configs)
        self.steps = configs.steps if 'steps' in configs else 1000
        self.cat_normalizer = cat_normalizer
        self.cf = None

    def forward(self):
        if self.cf is None:
            raise NotImplementedError('cf has not been initialized...')
        cf = self.cf * 1.0
        return cf if self.cat_normalizer is None else self.cat_normalizer.normalize(cf)

    def configure_optimizers(self):
        return torch.optim.RMSprop([self.cf], lr=0.001)

    def _loss_functions(self, x, c):
        # target
        y_pred = self.pred_fn(x)
        y_prime = flip_binary(y_pred)

        c_y = self.pred_fn(c)
        l_1 = F.binary_cross_entropy(c_y, y_prime)
        l_2 = F.mse_loss(x, c)
        return l_1, l_2

    def _loss_compute(self, l_1, l_2):
        return 1.0 * l_1 + 0.5 * l_2

    def generate_cf(self, x: torch.Tensor):
        self.cf = nn.Parameter(x.clone())
        optim = self.configure_optimizers()
        for i in range(self.steps):
            c = self()
            l_1, l_2 = self._loss_functions(x, c)
            loss = self._loss_compute(l_1, l_2)
            optim.zero_grad()
            loss.backward()
            optim.step()

        cf = self.cf * 1.0
        return self.cat_normalizer.normalize(cf, hard=True)